In [ ]:
import os
from dotenv import load_dotenv
from intake import open_catalog
import intake

import pandas as pd
import numpy as np
import geopandas as gpd

import datetime
from pystac_client import Client as psc

#import dask.distributed
import stackstac

from pyproj import CRS
from pyproj import Transformer

In [ ]:
# Variables à changer pour récupérer les données sources de la chaîne des feux dans le système de l'oeil
# Utiliser intake et dotenv 
# Intake pour ouvrir le catalogue à partir de la db postgres de l'OEIL
# dotenv pour gérer les credentials et les chemins d'accès

PATH_GEOM = r"E:\FILES\OEIL\datas\sentinel_surfaces_detectees_sept_oct_2023.gpkg"

URL = "https://earth-search.aws.element84.com/v1"



crs_rgnc = CRS.from_epsg(3163)
crs_4326 = CRS.from_epsg(4326)
transformer = Transformer.from_crs(crs_rgnc, crs_4326)

# On lit le fichier des surfaces brûlées dans un geodataframe
ba_test = gpd.read_file(PATH_GEOM)

# On créer une nouvelle colonne pour avoir un datetime de la date
ba_test['date_']= pd.to_datetime(ba_test['date'], format='%Y-%m-%d').dt.date

# On passe de multipolygon à polygon.
ba_test = ba_test.explode()
ba_test.head(2)

In [ ]:
# On établit la liste des communes de notre geodataframe
liste_commune = list(set(ba_test["commune"]))

# On récupère les surfaces_id qui ont été photo interprétés, fournis par Oriane
surfaces_id = {
    "burned" : [358032, 358018, 358010, 359919, 359594, 359614, 358008, 359524, 359592, 359944],
    "unburned"  : [357997, 358001, 358002, 358017, 358012, 359543, 359788, 359498, 359545, 360203],
    "doubt": [358026, 358033,359595, 359964, 362134, 362171, 362192, 362851, 360718, 359666]
}

# On prend un intervalle de temps avec 120 jours avant la première date de détection de surface brûlée du geopackage des surfaces brûlées
datemin = (min(ba_test['date_']) - datetime.timedelta(days=120)).strftime('%Y-%m-%d') 
datemax = max(ba_test['date_']).strftime('%Y-%m-%d')

ba_test_filter = ba_test[ba_test["commune"]=="LIFOU"]
bbox = ba_test_filter["geometry"].to_crs(4326).total_bounds

dates = f'{datemin}/{datemax}'
print(f'Emprise spatiale globale des formes identifiées {bbox}')

print(f'Interval temporel {dates}')

In [ ]:
client = psc.open(URL)
cc=40
search = client.search(
   
    collections=['sentinel-2-l2a'],
    bbox=bbox,
    datetime=dates,
    query={"eo:cloud_cover": {"lt": cc}}

    )

print(f"{search.matched()} scenes Sentinel-2 L2A trouvées dans l'interval temporel ayant - de {cc}% de couverture nuageuse")


In [ ]:
items = search.item_collection()

In [ ]:
stack = stackstac.stack(items)
print("stack.crs",stack.crs)
bbox[0]

In [ ]:
sentinel_stack = stackstac.stack(
                          items,
                          bounds_latlon=[bbox[0], bbox[1],  bbox[2],  bbox[3]],

                          gdal_env=stackstac.DEFAULT_GDAL_ENV.updated(
                               {'GDAL_HTTP_MAX_RETRY': 3,
                                'GDAL_HTTP_RETRY_DELAY': 5,
                               }),
                          epsg=4326
                          )

In [ ]:
data_indices = sentinel_stack.sel(band=["blue","rededge2", "rededge3", "green","red", "nir","nir08","swir22", "scl"]).to_dataset(dim='band')
#rgb = sentinel_stack.sel(band=["red", "green", "blue","scl"]).isel(time=1).compute()